In [157]:
import tensorflow as tf

In [158]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34, 31]

In [159]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [160]:
for sales in tf_dataset.as_numpy_iterator():
  print(sales)

21
22
-108
31
-1
32
34
31


In [161]:
for sales in tf_dataset.take(3):
  print(sales.numpy())

21
22
-108


In [162]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
  print(sales)

21
22
31
32
34
31


In [163]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
  print(sales)

1512
1584
2232
2304
2448
2232


In [164]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
  print(sales)

2232
1512
2304
2232
1584
2448


In [165]:
for sales_batch in tf_dataset.batch(2):
  print(sales_batch.numpy())

[2232 1512]
[2448 1584]
[2232 2304]


In [166]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
  print(sales)

[1512 2232]
[2304 2448]
[2232 1584]


In [167]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [168]:
images_ds = tf.data.Dataset.list_files('/content/drive/MyDrive/TF_pipeline_testpics/*/*', shuffle=False)


In [169]:
for file in images_ds.take(3):
  print(file.numpy())

b'/content/drive/MyDrive/TF_pipeline_testpics/cat pics/00tb-cats1-videoSixteenByNineJumbo1600.jpg'
b'/content/drive/MyDrive/TF_pipeline_testpics/cat pics/07CAT-STRIPES-mediumSquareAt3X-v2.jpg'
b'/content/drive/MyDrive/TF_pipeline_testpics/cat pics/1200px-Cat03.jpg'


In [170]:
images_ds = images_ds.shuffle(200)

for file in images_ds.take(3):
  print(file.numpy())

b'/content/drive/MyDrive/TF_pipeline_testpics/dog pics/neapolitan-mastiff-with-woman-on-leash-2-77685259-2000-81fba6fbcb8244d2adb24e31cc082c78.jpg'
b'/content/drive/MyDrive/TF_pipeline_testpics/dog pics/AdobeStock_94435211-500x500.jpg'
b'/content/drive/MyDrive/TF_pipeline_testpics/dog pics/GettyImages-1294547052.webp'


In [171]:
class_names = ["cat", "dog"]

In [172]:
image_count = len(images_ds)
image_count

197

In [173]:
train_size = int(image_count*0.8)

train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [174]:
len(train_ds)

157

In [175]:
len(test_ds)

40

In [176]:
s = '/content/drive/MyDrive/TF_pipeline_testpics/cat pics/00tb-cats1-videoSixteenByNineJumbo1600.jpg'

s.split("/")

['',
 'content',
 'drive',
 'MyDrive',
 'TF_pipeline_testpics',
 'cat pics',
 '00tb-cats1-videoSixteenByNineJumbo1600.jpg']

In [177]:
import os

In [178]:
def get_label(file_path):
  val = tf.strings.split(file_path, os.path.sep)[-2]
  if val == 'cat pics':
    return 'cat'
  else:
    return 'dog'

In [179]:
def process_image(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, [128, 128])

  return img, label

In [180]:
for t in train_ds.take(4):
  print(t.numpy())

b'/content/drive/MyDrive/TF_pipeline_testpics/cat pics/3683.jpg'
b'/content/drive/MyDrive/TF_pipeline_testpics/dog pics/800px-French_Bulldog_with_puppies.jpg'
b'/content/drive/MyDrive/TF_pipeline_testpics/cat pics/reddit-cats-judging-looks-00.jpg'
b'/content/drive/MyDrive/TF_pipeline_testpics/cat pics/kitten.jpg'


In [181]:
train_ds = train_ds.map(process_image)

for img, label in train_ds.take(3):
  print("Image", img)
  print("Label", label)

Image tf.Tensor(
[[[142.88214   122.88214    95.88214  ]
  [101.16693    81.16693    54.16693  ]
  [141.81268   121.81268    94.81268  ]
  ...
  [230.71497   210.71497   183.80872  ]
  [205.08832   182.09222   158.75629  ]
  [211.72272   189.72272   165.72272  ]]

 [[102.174255   82.174255   55.174255 ]
  [138.54205   118.54205    91.54205  ]
  [ 92.082825   72.082825   45.082825 ]
  ...
  [198.60638   177.65308   150.65308  ]
  [223.66986   207.49866   181.49866  ]
  [220.6189    202.6189    178.6189   ]]

 [[137.56305   117.56305    90.56305  ]
  [103.1012     83.1012     56.101196 ]
  [137.84888   117.84888    90.84888  ]
  ...
  [178.41473   153.7669    126.09082  ]
  [208.0744    189.65607   162.10938  ]
  [226.03119   206.80481   181.54694  ]]

 ...

 [[ 47.339844   68.66016    57.       ]
  [ 46.         66.67969    55.339844 ]
  [ 44.37079    68.27704    55.323914 ]
  ...
  [ 11.6796875  20.679688   15.6796875]
  [ 10.132996   19.132996   14.132996 ]
  [ 13.421814   19.421814  

In [182]:
def scale(image, label):
  return image/255, label

In [183]:
train_ds = train_ds.map(scale)
for image, label in train_ds.take(5):
  print("****Image: ", image.numpy()[0][0])
  print("****Label: ", label.numpy())

****Image:  [0.9019608 0.9372549 0.9647059]
****Label:  b'cat'
****Image:  [0.6095282  0.64090073 0.6918811 ]
****Label:  b'cat'
****Image:  [0.16285545 0.05305152 0.00108762]
****Label:  b'cat'
****Image:  [0.5749081 0.6807904 0.7239277]
****Label:  b'cat'
****Image:  [0.24387255 0.31446078 0.3615196 ]
****Label:  b'dog'
